In [14]:
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import pickle as pk
import re
from bs4 import BeautifulSoup
import spacy
import numpy as np

In [2]:
lemmatizer = WordNetLemmatizer()
print (lemmatizer.lemmatize('working', pos='v'))

work


In [3]:
def load_data(FILE_Path):
    otuput_sent = []
    lemmatizer = WordNetLemmatizer()
    with open(FILE_Path, "r") as f:
        train_tmp = f.read().split("\n\n")[:-1]
        for sample in train_tmp:
            sentence = re.split('\t|\n', sample)[1].replace('\"',"").replace(".","").replace(",","")\
            .replace("?","").replace("!","").replace(")","").replace("(","").replace("\'s","")\
            .replace("\'ve","").replace("\'t","").replace("\'re","").replace("\'d","")\
            .replace("\'ll","").replace("'","").replace(";","").replace(":","")
            answer = re.split('\t|\n', sample)[2]
            soup = BeautifulSoup(sentence,"lxml")

            e1 = str(soup.find('e1'))[4:-5]
            e2 = str(soup.find('e2'))[4:-5]
            word_list = sentence.split(" ")
            e1_check = 0
            e2_check = 0
            for word in word_list:
                if word.endswith("</e1>"):
                    e1_check = 1
                if word.endswith("</e2>"):
                    e2_check = 1
            if e1_check + e2_check != 2:
                print(sentence)
            for word in word_list:
                if "</e1>" in word:
                    a = word
                if "</e2>" in word:
                    b = word
            p1 = word_list.index(a)
            p2 = word_list.index(b)
            sentence = sentence.replace("<e1>","").replace("</e1>","").replace("<e2>","").\
            replace("</e2>","")
            sentence = [lemmatizer.lemmatize(word, pos='v') for word in sentence.split(" ")]
            sentence = " ".join(sentence)
            otuput_sent.append((sentence,e1,e2,p1,p2,answer))
    return otuput_sent

In [12]:
train_instance = load_data("./data/TRAIN_FILE.txt")
print("--------")
test_instance = load_data("./data/TEST_FILE_FULL.txt")
total = train_instance + test_instance

--------


In [5]:
len(test_instance)

2717

In [6]:
train_text = [ele[0] for ele in train_instance]

In [7]:
print(train_text[4783])
train_position[4783]

When thecoil of the relay be at rest not energize the common terminal 30 and the normally close terminal 87a have continuity


NameError: name 'train_position' is not defined

In [ ]:
train_position = [(ele[3],ele[4]) for ele in train_instance]

In [ ]:
# check p1 p2 order
for a, b in train_position:
    if a > b: 
        print(a,b)

In [ ]:
len(train_instance)

### depency parsing

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
train_text = [ele[0] for ele in train_instance]
test_test = [ele[0] for ele in test_instance]

In [ ]:
demo_id = 62
demo_sentence = train_text[demo_id]
print(demo_sentence)

doc = nlp(demo_sentence)

for pos, token in enumerate(doc):
    if pos in train_position[demo_id]:
        ancestors = [i for i in token.ancestors]
        print(token.text, token.tag_, token.dep_,
              token.shape_, token.is_stop, token.ent_type,ancestors,
              "lefts",[word.text for word in token.lefts],
              "rights",[word.text for word in token.rights],
              token.n_lefts,  # 2
            token.n_rights  # 1
             )

In [ ]:
train_mut_ancestors_list = []
train_dep_list = []
for ele in train_instance:
    sentence = ele[0]
    doc = nlp(sentence)
    entity = ele[1:3]
    both_ancestors = []
    both_dep = ""
    for pos, token in enumerate(doc):
        if pos in ele[3:5]:
            ancestors = [i.text for i in token.ancestors]
            both_ancestors.append(ancestors)
            both_dep+=token.dep_
    a = 0
    b = 0
    if entity[0] in both_ancestors[1]:
        a = 1
    elif entity[1] in both_ancestors[0]:
        b = 1
    mut_ancestors = [a,b]
    train_mut_ancestors_list.append(mut_ancestors)
    train_dep_list.append(both_dep)
    
test_mut_ancestors_list = []
test_dep_list = []
for ele in test_instance:
    sentence = ele[0]
    doc = nlp(sentence)
    entity = ele[1:3]
    both_ancestors = []
    both_dep = ""
    for pos, token in enumerate(doc):
        if pos in ele[3:5]:
            ancestors = [i.text for i in token.ancestors]
            both_ancestors.append(ancestors)
            both_dep+=token.dep_
    a = 0
    b = 0
    if entity[0] in both_ancestors[1]:
        a = 1
    elif entity[1] in both_ancestors[0]:
        b = 1
    mut_ancestors = [a,b]
    test_mut_ancestors_list.append(mut_ancestors)
    test_dep_list.append(both_dep)

In [ ]:
np.save("./features/train_mut_ancestors_list.npy", train_mut_ancestors_list)
np.save("./features/test_mut_ancestors_list.npy", test_mut_ancestors_list)

In [ ]:
a, b = np.unique(train_dep_list, return_counts=True)

In [ ]:
a_sorted = a[np.argsort(b)[::-1]]

In [ ]:
major_dep = a_sorted[:33]
major_dep

In [8]:
sorted(b,reverse=True)[:33]

NameError: name 'b' is not defined

In [9]:
train_dep_list_filter = []
for dep in train_dep_list:
    if dep in major_dep:
        train_dep_list_filter.append(dep)
    else:
        train_dep_list_filter.append("other")

test_dep_list_filter = []
for dep in test_dep_list:
    if dep in major_dep:
        test_dep_list_filter.append(dep)
    else:
        test_dep_list_filter.append("other")

NameError: name 'train_dep_list' is not defined

In [10]:
np.unique(test_dep_list_filter, return_counts=True)

NameError: name 'test_dep_list_filter' is not defined

In [171]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
enc = OneHotEncoder(sparse=False)
lb = LabelBinarizer()

In [188]:
# transform string to onehot
train_dep_list_filter = lb.fit_transform(train_dep_list_filter)
test_dep_list_filter = lb.transform(test_dep_list_filter)

In [189]:
np.save("./features/train_dep_list.npy", train_dep_list_filter)
np.save("./features/test_dep_list.npy", test_dep_list_filter)

In [192]:
train_dep_list_filter

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [191]:
test_dep_list_filter

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## WordNet hypernym

In [15]:
hyper_list = []
for idx in range(len(total)):
    entity_1 = total[idx][1]
    entity_2 = total[idx][2]
    
    synsets = wn.synsets(entity_1, pos=wn.NOUN)
    hyper_list_1 = []
    for syn in synsets:
        for hyper in syn.hypernyms():
            hyper_list_1 = hyper_list_1 + [x.lower() for x in hyper.lemma_names() if '_' not in x]
    
    synsets = wn.synsets(entity_2, pos=wn.NOUN)
    hyper_list_2 = []
    for syn in synsets:
        for hyper in syn.hypernyms():
            hyper_list_2 = hyper_list_2 + [x.lower() for x in hyper.lemma_names() if '_' not in x]
    hyper = ' '.join(list(set(hyper_list_1).union(set(hyper_list_2)))) 
    hyper_list.append(hyper)

In [16]:
train_hyper_list = hyper_list[:8000]
test_hyper_list = hyper_list[8000:]

In [17]:
np.save("./features/train_hyper_list.npy", train_hyper_list)
np.save("./features/test_hyper_list.npy", test_hyper_list)

In [18]:
total[1][1]

'child'

In [19]:
train_hyper_list[1]

'rootage descendant descendent somebody individual offspring mortal issue someone root soul trough juvenile progeny beginning birth person source origin'

## PropBank


In [20]:
with open('dict/propBank_dict.pk', 'rb') as f:
    propBank_dict = pk.load(f)

In [21]:
lmtzr = WordNetLemmatizer()
propBank_verb_entity_dict = {}
hyper_verb_list = []
for idx in range(len(total)):
    prop_rule = []
    temp_dict = {}
    for prop_rule in propBank_dict[idx]['propBank']:
        verb = prop_rule[0].split('.')[0]
        verb = lmtzr.lemmatize(verb,'v')
        entity = prop_rule[4]
        if len(entity.split(' ')) > 1:
            entity = entity.split(' ')[-1]
        #print(entity)
        if verb in temp_dict:
            temp_dict[verb].append(entity)
        else:
            temp_dict[verb] = [entity]
    
    temp_hyper_list = []
    for k, v in temp_dict.items():
        if total[idx][1] in v and total[idx][2] in v:
            synsets = wn.synsets(k, pos=wn.VERB)
            temp_hyper_list.append(k)

            for syn in synsets:
                if len(syn.hypernyms()) != 0:
                    temp_hyper_list = temp_hyper_list + [x.lower() for x in syn.hypernyms()[0].lemma_names() if '_' not in x]
            
    hyper = ' '.join(list(set(temp_hyper_list)))
    hyper_verb_list.append(hyper)

In [22]:
hyper_verb_list[:5]

['',
 'relate fix fasten stick adhere indispose restrain bond bind attach secure cover hold confine',
 'ingest use work have take act consume move exploit',
 '',
 '']

In [23]:
train_hyper_verb_list = hyper_verb_list[:8000]
test_hyper_verb_list = hyper_verb_list[8000:]

In [24]:
np.save("./features/train_hyper_verb_list.npy", train_hyper_verb_list)
np.save("./features/test_hyper_verb_list.npy", test_hyper_verb_list)